In [1]:
from __future__ import division
import numpy as np
import pyopencl as cl
import pyopencl.array

In [2]:
%load_ext pyopencl.ipython_ext


In [3]:
ctx = cl.create_some_context(interactive=True)
queue = cl.CommandQueue(ctx)

Choose platform:
[0] <pyopencl.Platform 'NVIDIA CUDA' at 0x12dcc40>
Choice [0]:0
Set the environment variable PYOPENCL_CTX='0' to avoid being asked again.


In [4]:
%%cl_kernel -o "-cl-fast-relaxed-math"

__kernel void sum_vector(__global const float *a,
__global const float *b, __global float *c)
{
  int gid = get_global_id(0);
  c[gid] = a[gid] + b[gid];
}

In [8]:
n = 1000000

a = cl.array.empty(queue, n, dtype=np.float32)
a.fill(15)

b_host = np.random.randn(n).astype(np.float32)
b = cl.array.to_device(queue, b_host)

c = cl.array.empty_like(a)

In [9]:
sum_vector(queue, (n,), None, a.data, b.data, c.data)

In [10]:
assert (c.get() == b_host + 15).all()
